In [2]:


from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [ ]:
!pip install librosa soundfile numpy sklearn pyaudio

In [ ]:
!pip install soundfile

In [ ]:
!pip install -q gradio

In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import gradio as gr
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [9]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [10]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [51]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('/content/drive/My Drive/content/Ravdess/Actor_*/*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.82,random_state=9)

In [52]:
# Split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.18)

In [53]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1180, 260)


In [54]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [55]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [56]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [57]:
# Predict for the test set
y_pred=model.predict(x_test)

In [58]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 57.31%


In [59]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.74      0.62      0.68        32
        calm       0.55      0.65      0.59        37
     disgust       0.70      0.68      0.69        38
     fearful       0.75      0.51      0.61        35
       happy       0.46      0.50      0.48        32
     neutral       0.45      0.29      0.36        17
         sad       0.38      0.55      0.45        33
   surprised       0.63      0.61      0.62        36

    accuracy                           0.57       260
   macro avg       0.58      0.55      0.56       260
weighted avg       0.59      0.57      0.58       260



In [60]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[20  0  3  1  4  0  1  3]
 [ 0 24  2  0  0  3  7  1]
 [ 1  3 26  1  3  1  1  2]
 [ 0  1  0 18  5  1  9  1]
 [ 3  2  2  2 16  0  3  4]
 [ 0  4  0  0  1  5  7  0]
 [ 2  8  0  0  2  1 18  2]
 [ 1  2  4  2  4  0  1 22]]
